## Parallel usage

This notebook explains how to use `parafields` in parallel. All features explained in the [Sequential Usage notebook](sequential.ipynb) are also valid in the parallel case. In order to run `parafields` in parallel, you need have installed it from source according to the installation instructions. Then, you need to initialize `mpi4py`:

In [ ]:
from mpi4py import MPI
import parafields

If you have not worked with `mpi4py`, it might be worthwhile to check their [beginner tutorial](https://mpi4py.readthedocs.io/en/stable/tutorial.html#running-python-scripts-with-mpi). The minimum take away message is that in order to run your code in parallel, you need to invoke the Python interpreter through `mpiexec`, e.g. like:

```
mpiexec -n 4 python yourscript.py
```

If you do not do this, your code will run sequentially.

### Passing communicators

By default, `parafields` uses `MPI.COMM_WORLD` as the communicator. If you want to use a different one, you can create it in `mpi4py` and pass it to the `generate_field` function:

In [ ]:
field = parafields.generate_field(comm=MPI.COMM_SELF)

### Data Distribution

The process of how `parafields` distributes the data to processors can be customized by passing the `partitioning` argument to `generate_field`. You can provide on of two things to `partitioning`:

* A tuple of integers whose length is the domain dimensions. The product of all entries must match the number of processors. E.g. to distribute to 8 processors using a 2x2x2 cube topology, you need to pass `(2, 2, 2)`.
* A function that accepts the number of processors and the resolution tuple as arguments and returns such tuple.

This is an example of such function that generates a striped topology:

In [ ]:
def striped_partitioning(P, cells):
    result = [1] * len(cells)
    result[0] = P
    return tuple(result)

It should be noted that there are some constraints to the partitioning process that arise from the internal workings of the FFTW library:

* TODO: Write this